# Ranking Toxic Comments using LSTM network maybe?

In [3]:
# Import
import tensorflow as tf
import pandas as pd
import numpy as np

My plan is to use... maybe two LSTM networks, to encode both comments and output a one-hot vector with 2 values, one for if it's greater and one for if it's less. Given that the dataset only identifies if one is greater than the other, we can only make determinations based on that, so the network reflects that

Something like this:



But first we need to download and process the data. Outside of python modules this does require `unzip`

In [10]:
# Download and extract data (run if on Linux/MacOS)
! rm -rf data
! kaggle competitions download --force -c jigsaw-toxic-severity-rating
! unzip jigsaw-toxic-severity-rating.zip -d data
! ls
! ls data

 89%|█████████████████████████████████▉    | 6.00M/6.72M [00:00<00:00, 14.9MB/s]
100%|██████████████████████████████████████| 6.72M/6.72M [00:00<00:00, 16.2MB/s]
Archive:  jigsaw-toxic-severity-rating.zip
  inflating: data/comments_to_score.csv  
  inflating: data/sample_submission.csv  
  inflating: data/validation_data.csv  
data                             requirements.txt
env                              toxic-lstm.ipynb
jigsaw-toxic-severity-rating.zip toxiccomments.ipynb
comments_to_score.csv sample_submission.csv validation_data.csv


Now that we have our data, we can create a function to pull it and convert it into a tensorflow dataset for our model. The worker is the id of the person that scored the comments. They're not really important so we can ignore that column. So for each row, we'll need the row as is and also the reverse of the row (a,b and b,a). This way we can train the model to understand both greater than and less than cases. Otherwise all of these outputs will be just 1. Prior experience has taught me that word based encoding is not all that useful. We'll encode using characters.

In [49]:
def input_data():
    """
    Extract and preprocess data for trainer
    """
    # Pull data from csv
    csv_data = pd.read_csv('data/validation_data.csv')
    csv_data = csv_data[['less_toxic', 'more_toxic']]
    
    # Create two sets with greater and lesser labels, concatenate and shuffle
    labeled_data_greater = csv_data[['less_toxic', 'more_toxic']].copy()
    labeled_data_greater.rename(columns={ 'less_toxic': 'seq_1', 'more_toxic': 'seq_2' }, inplace=True)
    labeled_data_greater['label'] = 'greater'
    labeled_data_less = csv_data[['more_toxic', 'less_toxic']].copy()
    labeled_data_less.rename(columns={ 'more_toxic': 'seq_1', 'less_toxic': 'seq_2' }, inplace=True)
    labeled_data_less['label'] = 'less'
    labeled_data = pd.concat([ labeled_data_greater, labeled_data_less ])
    labeled_data = labeled_data.sample(frac=1)
    
    # Now we take all sequences of characters and convert them to sequences 
    # of integers. We can do that using keras's StringLookup preprocessing 
    # layer, Which will take a string and spit out an array of integers 
    # encoding the characters of the string. This layer will need to scan 
    # the dataset to determine the appropriate encoding vocabulary for the 
    # characters. Since both sequences essentially contain all of the data, 
    # we can just use one of the sequences for the StringLookup to scan
    sequence_1 = tf.strings.unicode_split(labeled_data['seq_1'], input_encoding='UTF-8')
    print(sequence_1)

input_data()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

